### CNN for DDoS intrusion detection

In [13]:
import numpy as np
import pandas as pd

from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Sequential

from keras import callbacks
from keras.callbacks import CSVLogger

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
tf.logging.set_verbosity(tf.logging.ERROR)

Read and prepare dataset

In [4]:
np.random.seed(667)  # randomize

testdata = pd.read_csv("DDoS2007_spounged.csv", header=None)


#X = traindata.iloc[:,0:42]
#Y = traindata.iloc[:,0]
y_ini = testdata.iloc[:,0]
X_ini = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X_ini)
X_tras = scaler.transform(X_ini) #X Transformed

# reshape input to be [samples, time steps, features]
X = np.reshape(X_tras, (X_tras.shape[0],X_tras.shape[1],1))
y = np.array(y_ini)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

CNN Structure

In [8]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=3, strides=1, padding="same",activation="relu",input_shape=(41, 1)))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=(2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

Training (FIT)

In [10]:
cnn.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

Train on 700000 samples, validate on 300001 samples
Epoch 1/3
700000/700000 [==============================] - 512s 731us/step - loss: 0.4593 - acc: 0.7794 - val_loss: 0.4053 - val_acc: 0.8165
Epoch 2/3
700000/700000 [==============================] - 4861s 7ms/step - loss: 0.3972 - acc: 0.8195 - val_loss: 0.3603 - val_acc: 0.8377
Epoch 3/3
700000/700000 [==============================] - 498s 711us/step - loss: 0.3698 - acc: 0.8373 - val_loss: 0.3427 - val_acc: 0.8460


Results

In [15]:
y_prob = cnn.predict(X_test)
y_pred = (y_prob > 0.5).astype(np.int)
np.savetxt("cnn.txt", y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")

print("confusion matrix")
print("----------------------------------------------")
print("accuracy %.4f" %accuracy)
print("recall %.4f" %recall)
print("precision %.4f" %precision)
print("f1score %.4f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("confusion matrix", cm)
print("----------------------------------------------")

confusion matrix
----------------------------------------------
accuracy 0.8460
recall 0.6311
precision 0.8972
f1score 0.7410
confusion matrix [[187743   7568]
 [ 38622  66068]]
----------------------------------------------
